# Step 3: Creating the model inference script
To submit your algorithm to the challenge, you need to create an inference docker container. 

In [ ]:
# install pathology-whole-slide-data if needed
!pip3 install git+https://github.com/DIAGNijmegen/pathology-whole-slide-data@main

In [ ]:
# install detectron2 if needed
!pip3 install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

In [ ]:
!pip3 install creationism

In [ ]:
import os
import json
import torch
import creationism
from tqdm import tqdm



from wholeslidedata.interoperability.asap.annotationwriter import write_point_set
from wholeslidedata.image.wholeslideimage import WholeSlideImage
from wholeslidedata.iterators import create_patch_iterator, PatchConfiguration
from wholeslidedata.annotation.labels import Label

from utils.wsdetectron2 import Detectron2DetectionPredictor
from utils.structures import Point

Setting up the paths.

In [ ]:
image_path = r'./data/images/DI_S02_P000001.tif'
mask_path = r'./data/ROI_masks_025/DI_S02_P000001_mask.tif'
output_path = r"./outputs/results"
if not(os.path.isdir(output_path)): os.mkdir (output_path) 
json_filename = "detected-lymphocytes.json"

print(f"Pytorch GPU available: {torch.cuda.is_available()}")
print(image_path, mask_path)

Defining patch configuration for each image.

In [ ]:
patch_shape=(128,128,3)
spacings=(0.5,)
overlap=(0,0)
offset=(0,0)
center=False

patch_configuration = PatchConfiguration(patch_shape=patch_shape,
                                         spacings=spacings,
                                         overlap=overlap,
                                         offset=offset,
                                         center=center)

Loading the saved model.

In [ ]:
model = Detectron2DetectionPredictor(
    output_dir=output_path,
    threshold= 0.1,
    nms_threshold=0.3,
    weight_root = "./outputs/model_final.pth"
)

Creating a patch iterator using the roi mask and sliding windows.

In [ ]:
iterator = create_patch_iterator(image_path=image_path,
                               mask_path=mask_path,
                               patch_configuration=patch_configuration,
                               cpus=4,
                               backend='asap')

Some useful functions.

In [ ]:
def px_to_mm(px: int, spacing: float):
    return px * spacing / 1000

def to_wsd(points):
    """Convert list of coordinates into WSD points"""
    new_points = []
    for i, point in enumerate(points):
        p = Point(
            index=i,
            label=Label("lymphocyte", 1, color="blue"),
            coordinates=[point],
        )
        new_points.append(p)
    return new_points

def write_json_file(*, location, content):
    # Writes a json file
    with open(location, 'w') as f:
        f.write(json.dumps(content, indent=4))

Run inference on an image with loaded model.

In [ ]:
def inference(iterator, predictor, spacing, image_path, output_path, json_filename):
    print("predicting...")
    output_dict = {
        "name": "lymphocytes",
        "type": "Multiple points",
        "version": {"major": 1, "minor": 0},
        "points": [],
    }

    annotations = []
    counter = 0
    
    spacing_min = 0.25
    ratio = spacing/spacing_min
    with WholeSlideImage(image_path) as wsi:
        spacing = wsi.get_real_spacing(spacing_min)


    for x_batch, y_batch, info in tqdm(iterator):
        x_batch = x_batch.squeeze(0)
        y_batch = y_batch.squeeze(0)

        predictions = predictor.predict_on_batch(x_batch)
        for idx, prediction in enumerate(predictions):

            c = info['x']
            r = info['y']

            for detections in prediction:
                x, y, label, confidence = detections.values()

                if x == 128 or y == 128:
                    continue

                if y_batch[idx][y][x] == 0:
                    continue
                
                x = x*ratio + c # x is in spacing= 0.5 but c is in spacing = 0.25
                y= y*ratio + r
                prediction_record = {
                    "name" : "Point "+str(counter),
                    "point": [
                        px_to_mm(x, spacing),
                        px_to_mm(y, spacing),
                        0.24199951445730394,
                    ],
                    "probability": confidence,
                }
                output_dict["points"].append(prediction_record)
                annotations.append((x, y))
                counter += 1



    print(f"Predicted {len(annotations)} points")
    print("saving predictions...")

    # saving xml file
    annotations_wsd = to_wsd(annotations)
    xml_filename = 'points_results.xml'
    output_path_xml = os.path.join(output_path,xml_filename)
    write_point_set(
        annotations_wsd,
        output_path_xml,
        label_color="blue",
    )

        
    # saving json file
    output_path_json = os.path.join(output_path, json_filename)
    write_json_file(
        location=output_path_json,
        content=output_dict
    )

    print("finished!")

In [ ]:
inference(
    iterator=iterator,
    predictor=model,
    spacing = spacings[0],
    image_path=image_path,
    output_path=output_path,
    json_filename=json_filename
)

iterator.stop()